In [9]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
import math
import sympy
import scipy
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV, LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
import collections
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix
from sklearn.cross_decomposition import PLSRegression, PLSSVD
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.decomposition import PCA
import collections
from collections import OrderedDict
import xgboost as xgb
from mlxtend.regressor import StackingRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor, XGBClassifier
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [10]:
path = "D:\Kevin Liang/Documents/1_UT_SENIOR/UT_AUSTIN_FALL_2017/EE_379K/Kaggle_Comp_Midterm/"

In [11]:
X_final = pd.read_csv(path + "test_final.csv", header = 0)
train_data = pd.read_csv(path + "train_final.csv", header = 0)

# retrieve ID column for creating submission CSV
ID = X_final["id"]

# create X test
X_final = X_final.drop(["id"], axis = 1)
    
# replace NaN with mean of column
train_data = train_data.fillna(train_data.mean())
X_final = X_final.fillna(train_data.mean())

# create X train and Y train
X = train_data.drop(["Y","id"], axis = 1)
y = train_data.loc[:,"Y"]

# calculated correlation matrix to find correlations between features
data_corr = train_data.corr()

#for col_name in X.columns:
#    X = remove_outliers(data = X, column = str(col_name), mean = X[col_name].mean(), std = X[col_name].std())

#print data_corr
#for x in train_data.columns:
#    sns.distplot(train_data[x])
#    sns.plt.show()

In [12]:
# XGBoost from previous lab

parameters = {
    'max_depth':[1, 2, 6, 8, 10],
    'n_estimators':[25, 50, 100, 150, 200],
    'reg_alpha' : [0.001, 0.05, 0.1, 0.25, 0.5]
}

xgmodel = xgb.XGBRegressor()
#xg_clf = GridSearchCV(xgmodel, parameters, cv = 5, scoring = "roc_auc").fit(X, y)

cv_score = cross_val_score(xgmodel, X, y, cv=5, scoring = "roc_auc").mean()
print "XGBoost Regressor Score: " + str(cv_score)

xgmodel = xgmodel.fit(X,y)
y_pred = xgmodel.predict(X_final)

# public lb 0.86326
# local roc auc 0.858436561237

XGBoost Regressor Score: 0.859195005372


In [13]:
# Ridge
alphas = [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75, 100,150,200,210,220,230,240,250,300,400]
r_cv = RidgeCV(alphas = alphas)
cv_score = cross_val_score(r_cv, X, y, cv=5, scoring = "roc_auc").mean()
print "RidgeCV Score: " + str(cv_score)
r_cv = r_cv.fit(X, y)
print "Optimal RidgeCV alpha: " + str(r_cv.alpha_) + '\n'
y_pred = r_cv.predict(X_final)

# Lasso
alphas = [10, 1, 0.1, 0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005]
l_cv = LassoCV(alphas = alphas)
cv_score = cross_val_score(l_cv, X, y, cv=5, scoring = "roc_auc").mean()
print "LassoCV Score: " + str(cv_score)
l_cv = l_cv.fit(X, y)
print "Optimal LassoCV alpha: " + str(r_cv.alpha_) + '\n'
y_pred = l_cv.predict(X_final)

# Logistic 
lin_cv = LinearRegression()
cv_score = cross_val_score(lin_cv, X, y, cv=10, scoring = "roc_auc").mean()
print "LinearRegression Score: " + str(cv_score) + '\n'
lin_cv = lin_cv.fit(X,y)
y_pred = lin_cv.predict(X_final)

log_cv = LogisticRegression()
cv_score = cross_val_score(log_cv, X, y, cv=10, scoring = "roc_auc").mean()
print "LogisticRegression Score: " + str(cv_score) + '\n'
lin_cv = log_cv.fit(X,y)
y_pred = log_cv.predict(X_final)


RidgeCV Score: 0.688477725342
Optimal RidgeCV alpha: 150.0

LassoCV Score: 0.688909950315
Optimal LassoCV alpha: 150.0

LinearRegression Score: 0.687750320969

LogisticRegression Score: 0.634608940378



In [14]:
rf_reg = RandomForestRegressor(random_state = 12323)
cv_score = cross_val_score(rf_reg, X, y, cv=10, scoring = "roc_auc").mean()
print "RandomForestRegressor CV Score: " + str(cv_score) + '\n'

xg_reg = xgb.XGBRegressor()
cv_score = cross_val_score(xg_reg, X, y, cv=10, scoring = "roc_auc").mean()
print "XBGRegressor CV Score: " + str(cv_score)

RandomForestRegressor CV Score: 0.771096376759

XBGRegressor CV Score: 0.858935216142


In [15]:
# PCR (PCA + linearRegression)

components = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]

component = 0
total_variance = 0
cvScore = 0
print "# of Components / Negative Cross Val Score / Variance"
for x in components:
    pca = PCA(n_components = x, svd_solver = "full")
    cv_score = cross_val_score(pca, X, y, cv=10).mean()
    pca  = pca.fit(X, y)
    #print x, cv_score, pca.explained_variance_ratio_[-1]
    if total_variance < .9:
        total_variance += pca.explained_variance_ratio_[-1]
        component = x
        cvScore = cv_score

for x in components:
    r_cv = xgb.XGBRegressor()
    r_cv = r_cv.fit(X.iloc[:,:x+1], y)
    X_test_pcr = X_final.iloc[:,:x+1]
    y_pred = r_cv.predict(X_test_pcr)
    cv_score = cross_val_score(r_cv, X.iloc[:,:x+1], y, cv=5, scoring = "roc_auc").mean()
    print "Number of Components:" + str(x)
    print "ROC AUC Score: " + str(cv_score) +'\n'



# of Components / Negative Cross Val Score / Variance
Number of Components:1
ROC AUC Score: 0.617352506592

Number of Components:2
ROC AUC Score: 0.799235945921

Number of Components:3
ROC AUC Score: 0.799484327524

Number of Components:4
ROC AUC Score: 0.803292533717

Number of Components:5
ROC AUC Score: 0.802676750754

Number of Components:6
ROC AUC Score: 0.802821042899

Number of Components:7
ROC AUC Score: 0.802502924668

Number of Components:8
ROC AUC Score: 0.802878454172

Number of Components:9
ROC AUC Score: 0.806772849115

Number of Components:10
ROC AUC Score: 0.806862892876

Number of Components:11
ROC AUC Score: 0.806690189882

Number of Components:12
ROC AUC Score: 0.806368549663

Number of Components:13
ROC AUC Score: 0.830532502471

Number of Components:14
ROC AUC Score: 0.830343718606

Number of Components:15
ROC AUC Score: 0.829968971047

Number of Components:16
ROC AUC Score: 0.829912835948

Number of Components:17
ROC AUC Score: 0.834043018217

Number of Components

In [16]:
# stacking attempt lol (failed)

alphas = [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75] 
r_cv = RidgeCV(alphas = alphas)
r_cv = r_cv.fit(X, y)
y_pred_R = r_cv.predict(X_final)

alphas = [10, 1, 0.1, 0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005]
l_cv = LassoCV(alphas = alphas)
l_cv = l_cv.fit(X, y)
y_pred_L = l_cv.predict(X_final)

X_final['Ridge Predictions'] = pd.Series(y_pred_R, index=X_final.index)
X_final['Lasso Predictions'] = pd.Series(y_pred_L, index= X_final.index)
y_pred_R = y_pred_R[:len(y_pred_R) -2:]
y_pred_L = y_pred_L[:len(y_pred_L) -2:]
X['Ridge Predictions'] = pd.Series(y_pred_R, index = X.index)
X['Lasso Predictions'] = pd.Series(y_pred_L, index = X.index)

        

In [17]:
param_test1 = {
    'scale_pos_weight':[0.1,0.5,1,2,3]
    
}
gsearch1 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.01, n_estimators=1100, max_depth=3,
 min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27, base_score = 0.3, max_delta_step = 1), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch1.fit(X,y)
y_pred = gsearch1.predict(X_final)
print gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_


#{'max_depth': 3, 'min_child_weight': 4},
#0.86023918063345839

#{'max_depth': 3, 'min_child_weight': 3},
# 0.86080023248484605)

#{'subsample': 0.8, 'colsample_bytree': 0.8}
# 0.861038005534

#{'reg_alpha': 1e-05} 
#0.86103803692

#{'n_estimators': 110} 
#0.861092907843

#{'n_estimators': 120, 'reg_alpha': 0.001} 0.837285316704

#{'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 3} 0.861206345813

#{'n_estimators': 1100, 'learning_rate': 0.01} 0.861236793201

#{'scale_pos_weight': 2} 0.861321702375

[mean: 0.86061, std: 0.00853, params: {'scale_pos_weight': 0.1}, mean: 0.86077, std: 0.00843, params: {'scale_pos_weight': 0.5}, mean: 0.86100, std: 0.00844, params: {'scale_pos_weight': 1}, mean: 0.86117, std: 0.00873, params: {'scale_pos_weight': 2}, mean: 0.86119, std: 0.00865, params: {'scale_pos_weight': 3}] {'scale_pos_weight': 3} 0.861192713358


I used this one cell to tune the hyperparameters for the XGBRegressor. I would run with set hyperparameters and record the local CV score. I would then tune the parameters and run others. I only recorded scores that imporved previous scores which are shown in the comments of the code. 

In [18]:
# XGBClassifier
param_test2 = {
 'reg_alpha':[1]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.01, n_estimators=1000, max_depth=3,
 min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27,reg_alpha = 1), 
                       param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch2.fit(X,y)
y_pred = gsearch2.predict(X_final)
print gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

# {'max_depth': 2, 'min_child_weight': 3} 0.8572636502
# {'n_estimators': 120} 0.860541566744

[mean: 0.86105, std: 0.00769, params: {'reg_alpha': 1}] {'reg_alpha': 1} 0.861052729409


In [19]:
# Stacking with StackingRegressor
r_alphas = [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75] 
l_alphas = [10, 1, 0.1, 0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005]
lr = LinearRegression()
logr = LogisticRegression()
r_cv = RidgeCV(alphas = r_alphas)
l_cv = LassoCV(alphas = l_alphas)
xgb1 = XGBRegressor(seed = 0, nthread = -1, colsample_bytree = 0.69, subsample = 0.69, learning_rate = 0.1, 
        max_depth = 6, min_child_weight = 1, n_estimators = 100)
xgb2 = ExtraTreesRegressor(random_state = 0, n_jobs = -1, n_estimators = 100, 
        max_features = 0.49, max_depth = 8, min_samples_leaf = 2)
xgb3 = RandomForestRegressor(random_state = 0, n_jobs = -1, n_estimators = 100, 
        max_features = 0.22, max_depth = 8, min_samples_leaf = 2)
lin_r = XGBRegressor(seed = 0, nthread = -1, colsample_bytree = 0.69, subsample = 0.69, learning_rate = 0.1, 
        max_depth = 6, min_child_weight = 1, n_estimators = 100)

stregr = StackingRegressor(regressors=[lr, logr, r_cv, l_cv, xgb1, xgb2, xgb3], 
                           meta_regressor=lin_r)
print "Cross Val Score(stacked): " + str(cross_val_score(stregr, X, y, cv=5, scoring = "roc_auc").mean()) + '\n'
y_pred = stregr.fit(X,y).predict(X_final)

Cross Val Score(stacked): 0.801978687251



In [20]:
#predictions = pd.DataFrame(y_pred, columns = ["id", "Y"])
#predictions.to_csv("predictions.csv")
predictions = pd.DataFrame({"id":ID, "Y":y_pred})
predictions = predictions[predictions.columns[::-1]]
predictions.to_csv("predictions.csv", index = False)